In [46]:
import grpc
import demo_pb2
import demo_pb2_grpc
from concurrent import futures
import subprocess
import os

In [28]:
import inspect, sys
from IPython.core.magics.code import extract_symbols

def new_getfile(object, _old_getfile=inspect.getfile):
    if not inspect.isclass(object):
        return _old_getfile(object)
    
    # Lookup by parent module (as in current inspect)
    if hasattr(object, '__module__'):
        object_ = sys.modules.get(object.__module__)
        if hasattr(object_, '__file__'):
            return object_.__file__
    
    # If parent module is __main__, lookup by methods (NEW)
    for name, member in inspect.getmembers(object):
        if inspect.isfunction(member) and object.__qualname__ + '.' + member.__name__ == member.__qualname__:
            return inspect.getfile(member)
    else:
        raise TypeError('Source for {!r} not found'.format(object))
inspect.getfile = new_getfile

def stringify_class(obj):
    cell_code = "".join(inspect.linecache.getlines(new_getfile(obj)))
    class_code = extract_symbols(cell_code, obj.__name__)[0][0]
    return class_code

In [58]:
class CheckoutService(demo_pb2_grpc.CheckoutServiceServicer):
    def PlaceOrder(self, request, context):
        order_result = demo_pb2.OrderResult(
            order_id='123',
            status='success'
        )
        response = demo_pb2.PlaceOrderResponse(
            order=order_result
        )
        return response

class ShippingService(demo_pb2_grpc.ShippingServiceServicer):
    def GetQuote(self, request, context):
        ...
    
    def ShipOrder(self, request, context):
        ...

class PaymentService(demo_pb2_grpc.PaymentServiceServicer):
    def Charge(self, request, context):
        ...

In [61]:
# TODO: random port numbers > 50000
def generate_grpc_server(server_obj):
    return f"""
import sys

import grpc
import demo_pb2
import demo_pb2_grpc
from concurrent import futures

# define the gRPC service class
{stringify_class(server_obj)}
class ServerControlService(demo_pb2_grpc.ServerControlService):
    def TerminateServer(self, request, context):
        if request.action == 'terminate':
            print("terminating server")
            server.stop(0)
        return demo_pb2.Empty()

# create a gRPC server and add the service to it
server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
demo_pb2_grpc.add_{server_obj.__name__}Servicer_to_server({server_obj.__name__}(), server)
demo_pb2_grpc.add_ServerControlServiceServicer_to_server(ServerControlService(), server)
server.add_insecure_port('[::]:50051')

# start the server
server.start()
print('Server started on port 50051')

# wait for the server to finish
server.wait_for_termination()
"""

In [59]:
def test_grpc():
    server_processes = []
    filenames = []
    for obj in [CheckoutService, ShippingService, PaymentService]:
        filename = f"{obj.__name__}.py"
        filenames.append(filename)
        with open(filename, 'w') as f:
            f.write(generate_grpc_server(obj))

        process = subprocess.Popen(["python", filename], stdout=subprocess.PIPE, stderr= subprocess.PIPE)
        server_processes.append(process)
        # stdout, stderr = process.communicate()

    # TODO: generate 
    stub_process = subprocess.Popen(["python", "grpc_client.py"], stdout=subprocess.PIPE, stderr= subprocess.PIPE)
    # stdout, stderr = stub_process.communicate()
    # TODO: testing logic
    
    for process in server_processes:
        stdout, stderr = process.communicate()
        print(stdout, stderr)
        
    for obj in [CheckoutService, ShippingService, PaymentService]:
        name = obj.__name__
        os.remove(f"{name}.py")

In [50]:
with open('payment_service.py', 'w') as f:
    f.write(checkout_service_code)

payment_process = subprocess.Popen(["python", "payment_service.py"], stdout=subprocess.PIPE, stderr= subprocess.PIPE)
subprocess.Popen(["python", "grpc_client.py"], stdout=subprocess.PIPE, stderr= subprocess.PIPE)
stdout, stderr = checkout_process.communicate()
print(stdout)
print(stderr)
os.remove('payment_service.py')

b'Server started on port 50051\nterminating server\n'
b''


In [62]:
test_grpc()

b'Server started on port 50051\nterminating server\n' b''


KeyboardInterrupt: 